In [1]:
import asyncio
from uiprotect import ProtectApiClient
import json

# UniFi Protect credentials
UNIFI_PROTECT_HOST = "192.168.40.5"
UNIFI_PROTECT_USER = "oskarwolf"
UNIFI_PROTECT_PASSWORD = "U@Os97KarWOLF"
VERIFY_SSL = False
UNIFI_PROTECT_PORT = 443

# Target camera names
target_names = ["Cam Cafe POS Shelves", "Cam Front Entrance"]

# Normalize the list for better comparison
target_names_normalized = [name.strip().lower() for name in target_names]

async def get_selected_camera_data():
    protect = ProtectApiClient(
        host=UNIFI_PROTECT_HOST,
        port=UNIFI_PROTECT_PORT,
        username=UNIFI_PROTECT_USER,
        password=UNIFI_PROTECT_PASSWORD,
        verify_ssl=VERIFY_SSL,
    )

    await protect.update()

    selected_data = []
    print("🔍 Checking cameras for matching names...\n")
    for cam in protect.bootstrap.cameras.values():
        cam_name_normalized = cam.name.strip().lower()
        if cam_name_normalized in target_names_normalized:
            print(f"✅ Matched: {cam.name}")
            channel = cam.channels[0] if cam.channels else None
            resolution = f"{channel.width}x{channel.height}" if channel else "N/A"

            selected_data.append({
                "Name": cam.name,
                "ID": cam.id,
                "Model": cam.model,
                "MAC": cam.mac,
                "Connected": cam.is_connected,
                "Recording": cam.is_recording,
                "Resolution": resolution,
                "Firmware": cam.firmware_version,
                "Last Seen": cam.last_seen.isoformat() if cam.last_seen else "N/A",
            })
        else:
            # Optional: show unmatched for debugging
            print(f"⛔ Not matched: {cam.name}")

    await protect.close_session()

    with open("../results/EDA/selected_camera_data.json", "w") as f:
        json.dump(selected_data, f, indent=4)

    print("\n📁 All matching camera data saved to 'selected_camera_data.json'")

await get_selected_camera_data()


🔍 Checking cameras for matching names...

⛔ Not matched: Cam Back Entrance
⛔ Not matched: Cam Parking Front
⛔ Not matched: Cam Hall Toilet
⛔ Not matched: Cam Parking Walker Drive
⛔ Not matched: Cam Office
⛔ Not matched: Cam Back Deliveries
⛔ Not matched: Cam House Garden Lawn
⛔ Not matched: Cam House Lawn Bedroom
⛔ Not matched: Cam Cafe Overhead
⛔ Not matched: Cam Pos Till2
⛔ Not matched: Cam Hall Corner
⛔ Not matched: Cam Toilets
⛔ Not matched: Cam Hall
⛔ Not matched: Cam Parking Entrance
⛔ Not matched: Cam Cafe Fridge
⛔ Not matched: Cam House Kitchen Door
⛔ Not matched: Cam Shed Tools
⛔ Not matched: Cam Cafe Store
⛔ Not matched: Cam Shed Road South
⛔ Not matched: Cam Office Money
⛔ Not matched: Cam Parking DA
⛔ Not matched: Cam Hall Outside seating
⛔ Not matched: Cam Back Bakkie
⛔ Not matched: Cam Shed Back 
✅ Matched: Cam Cafe POS Shelves
⛔ Not matched: Cam POS
⛔ Not matched: Cam Back Keith
⛔ Not matched: Cam Cafe Kitchen Production
⛔ Not matched: Cam Faulty
⛔ Not matched: G3 Flex
⛔

In [ ]:
import subprocess
from datetime import datetime, timedelta
import os

# ---- Configuration ----
USERNAME = "oskarwolf"
PASSWORD = "U@Os97KarWOLF"
HOST = "192.168.40.5"
PORT = "443"
CAMERA_MAC = "67cf009601ea2703e430e6a0"  # Cam Front Entrance MAC
NO_VERIFY = "--no-verify"
API_KEY = "uE_8WTRChZXsicPRx3nR22Y3Mq6_NdJA"

# ---- Output folder ----
CLIP_DIR = r"C:\Users\mrosk\OneDrive\Desktop\thesis_demographics_cv\data\raw\cafe_pos"
os.makedirs(CLIP_DIR, exist_ok=True)

# ---- Calculate yesterday's date ----
yesterday = datetime.utcnow().date() - timedelta(days=2)
base_time = datetime.combine(yesterday, datetime.min.time()).replace(hour=8, minute=00)

# ---- Loop from 08:30 to 17:00 in 1-hour intervals ----
for i in range(9):  # 9 hourly clips
    start_time = base_time + timedelta(hours=i)
    end_time = start_time + timedelta(hours=1)

    start_str = start_time.strftime("%Y-%m-%dT%H:%M:%S")
    end_str = end_time.strftime("%Y-%m-%dT%H:%M:%S")
    filename = f"CafePOS_{start_time.strftime('%Y%m%d_%H%M')}_to_{end_time.strftime('%H%M')}.mp4"
    output_path = os.path.join(CLIP_DIR, filename)

    command = [
        "uiprotect",
        "-U", USERNAME,
        "-P", PASSWORD,
        "-a", HOST,
        "-p", PORT,
        "--api-key", API_KEY,
        NO_VERIFY,
        "cameras",
        CAMERA_MAC,
        "save-video",
        output_path,
        start_str,
        end_str,
    ]

    print(f"\n🎬 Downloading: {filename}")
    try:
        subprocess.run(command, check=True)
        print("✅ Success.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to download {filename}:\n{e}")



🎬 Downloading: CafePOS_20251107_1600_to_1700.mp4
✅ Success.

🎬 Downloading: CafePOS_20251107_1700_to_1800.mp4
